In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/train.csv')
print([str(x) for x in df.iloc[0:0,:]])
df2 = pd.read_csv('/kaggle/input/nexus-by-djs-nsdc-ultraceuticals/test.csv')

In [ ]:
from scipy.stats import chi2_contingency
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm

In [ ]:
# def chi_squared_for_pairs(df, variables):
#     results = {}
#     for i in range(len(variables)):
#         for j in range(i + 1, len(variables)):
#             var1 = variables[i]
#             var2 = variables[j]
#             contingency_table = pd.crosstab(df[var1], df[var2])
#             chi2, p, dof, expected = chi2_contingency(contingency_table)
#             results[(var1, var2)] = {'chi2': chi2, 'p-value': p}
#     return results

# categorical_vars = ['ID', 'TargetID', 'DRUGID', 
#                     'DRUGTYPE', 'Drug_high_status', 'DRUGNAME',
#                     'PUBCHCID', 'Disease_of_highest_status', 
#                     'Drug_Status', 'UNIPROID', 'TARGNAME', 
#                     'GENENAME', 'SYNONYMS', 'FUNCTION', 
#                     'BIOCLASS', 'SEQUENCE', 'Disease', 
#                     'Accession Number', 'Target_Status']

# # Perform Chi-Squared tests for all pairs
# chi_squared_results = chi_squared_for_pairs(df, categorical_vars)

# for vars, result in chi_squared_results.items():
#     if result['p-value']>0:
#         print(f"Variables: {vars}, Chi-squared: {result['chi2']:.2f}, P-value: {result['p-value']:.3f}")


In [ ]:
categorical_vars = ['ID', 'TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 
                    'DRUGNAME', 'PUBCHCID', 'Disease_of_highest_status', 
                    'Drug_Status', 'UNIPROID', 'TARGNAME', 'GENENAME', 
                    'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE', 
                    'Disease', 'Accession Number']

In [ ]:
def chi_squared_with_target_status(df, target_variable, variables):
    results = {}
    for var in variables:
        contingency_table = pd.crosstab(df[var], df[target_variable])
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        results[var] = {'chi2': chi2, 'p-value': p}
    return results

# List of categorical variables excluding 'Target_Status'
# Perform Chi-Squared tests for all specified variables against 'Target_Status'
target_variable = 'Target_Status'
chi_squared_results = chi_squared_with_target_status(df, target_variable, categorical_vars)

# Print the results
for var, result in chi_squared_results.items():
    print(f"Variable: {var}, Chi-squared: {result['chi2']:.2f}, P-value: {result['p-value']:.3f}")


In [ ]:
X_train = df[categorical_vars]
X_train_encoded = pd.get_dummies(X_train, drop_first=True)

y_train = df['Target_Status']

X_test = pd.get_dummies(df2[categorical_vars], drop_first=True)
X_test_encoded = X_test.reindex(columns=X_train_encoded.columns, fill_value=0)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train_encoded, y_train)

y_pred = model.predict(X_test_encoded)

submission = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y_pred
})
submission.to_csv('submission1.csv', index=False)

In [ ]:
X2_train = df[categorical_vars[1:]]
y2_train = df['Target_Status']
label_encoder = LabelEncoder()
y2_train_numeric = label_encoder.fit_transform(y2_train)
X2_test = df2[categorical_vars[1:]]

encoder = ce.TargetEncoder(cols=categorical_vars[1:])
X2_train_encoded = encoder.fit_transform(X2_train, y2_train_numeric)
X2_test_encoded = encoder.transform(X2_test)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X2_train_encoded, y2_train_numeric)

y2_pred_numeric = model.predict(X2_test_encoded)
y2_pred = label_encoder.inverse_transform(y2_pred_numeric)

submission2 = pd.DataFrame({
    'ID': df2['ID'],
    'Prediction': y2_pred
})

submission2.to_csv('submission2.csv', index=False)

In [ ]:
main_vars = ['ID', 'TargetID', 'DRUGID',  
                    'DRUGNAME', 'PUBCHCID', 'UNIPROID', 'TARGNAME', 'GENENAME', 
                    'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE', 
                    'Disease', 'Accession Number']

In [ ]:
X3_train = df[main_vars[1:]]
y3_train = df['Target_Status']
label_encoder3 = LabelEncoder()

y3_train_numeric = label_encoder3.fit_transform(y3_train)
X3_test = df2[main_vars[1:]]

encoder = ce.TargetEncoder(cols=main_vars[1:])
X3_train_encoded = encoder.fit_transform(X3_train, y3_train_numeric)
X3_test_encoded = encoder.transform(X3_test)

In [ ]:
model3 = RandomForestClassifier(random_state=42)
model3.fit(X3_train_encoded, y3_train_numeric)

y3_pred_numeric = model3.predict(X3_test_encoded)
y3_pred = label_encoder3.inverse_transform(y3_pred_numeric)

submission3 = pd.DataFrame({
    'ID': df2['ID'],
    'Prediction': y3_pred
})

submission3.to_csv('submission3.csv', index=False)

In [ ]:
X4_train = df[main_vars]
X4_train_encoded = pd.get_dummies(X4_train, drop_first=True)

y4_train = df['Target_Status']

X4_test = pd.get_dummies(df2[main_vars], drop_first=True)
X4_test_encoded = X4_test.reindex(columns=X4_train_encoded.columns, fill_value=0)

In [ ]:
model4 = RandomForestClassifier(random_state=42)
model4.fit(X4_train_encoded, y4_train)

y4_pred = model4.predict(X4_test_encoded)

submission4 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y4_pred
})
submission4.to_csv('submission4.csv', index=False)

In [ ]:
class ProgressBar(RandomizedSearchCV):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pbar = None

    def fit(self, X, y=None, **fit_params):
        n_candidates = len(self.param_distributions)
        self.pbar = tqdm(total=n_candidates, desc="Randomized Search Progress")
        
        def callback(*args):
            self.pbar.update()

        super().fit(X, y, **fit_params)
        self.pbar.close()

In [ ]:
main_vars5 = ['ID', 
                    'DRUGNAME',  'TARGNAME', 'GENENAME', 
                    'SYNONYMS', 'FUNCTION', 'SEQUENCE', 
                    'Disease', 'Accession Number']

In [ ]:
X5_train = df[main_vars5]
X5_train_encoded = pd.get_dummies(X5_train, drop_first=True)

y5_train = df['Target_Status']

X5_test = pd.get_dummies(df2[main_vars5], drop_first=True)
X5_test_encoded = X5_test.reindex(columns=X5_train_encoded.columns, fill_value=0)

In [ ]:
model5_best = RandomForestClassifier(
    criterion='entropy',
    max_depth=12,
    max_features='log2',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=90,
    random_state=42,
    warm_start=True 
)

with tqdm(total=model5_best.n_estimators, desc="Fitting Trees") as pbar:
    for i in range(1, model5_best.n_estimators + 1):
        model5_best.set_params(n_estimators=i)
        model5_best.fit(X5_train_encoded, y5_train) 
        pbar.update(1) 
y5_pred = model5_best.predict(X5_test_encoded)

submission5 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y5_pred
})
submission5.to_csv('submission5.csv', index=False)

In [ ]:
main_vars6 = ['ID', 
                    'DRUGNAME',  'TARGNAME', 'GENENAME', 
                    'SYNONYMS', 'FUNCTION', 'SEQUENCE', 
                    'Disease', 'Accession Number']

In [ ]:
X6_train = df[main_vars6]
X6_train_encoded = pd.get_dummies(X6_train, drop_first=True)

y6_train = df['Target_Status']

X6_test = pd.get_dummies(df2[main_vars6], drop_first=True)
X6_test_encoded = X6_test.reindex(columns=X6_train_encoded.columns, fill_value=0)

In [ ]:
model6 = RandomForestClassifier(random_state=42)
model6.fit(X6_train_encoded, y6_train)

y6_pred = model6.predict(X6_test_encoded)

submission6 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y6_pred
})
submission6.to_csv('submission6.csv', index=False)

In [ ]:
main_vars7 = ['ID', 
                    'DRUGNAME',  'TARGNAME', 'GENENAME', 
                    'SYNONYMS','FUNCTION', 'SEQUENCE', 
                    'Disease']

In [ ]:
X7_train = df[main_vars7]
X7_train_encoded = pd.get_dummies(X7_train, drop_first=True)

y7_train = df['Target_Status']

X7_test = pd.get_dummies(df2[main_vars7], drop_first=True)
X7_test_encoded = X7_test.reindex(columns=X7_train_encoded.columns, fill_value=0)

In [ ]:
model7 = RandomForestClassifier(random_state=42)
model7.fit(X7_train_encoded, y7_train)

y7_pred = model7.predict(X7_test_encoded)

submission7 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y7_pred
})
submission7.to_csv('submission7.csv', index=False)

In [ ]:
main_vars8 = ['ID', 
                    'DRUGNAME',  'TARGNAME', 'GENENAME', 
                    'FUNCTION', 'SEQUENCE', 
                    'Disease']

In [ ]:
X8_train = df[main_vars8]
X8_train_encoded = pd.get_dummies(X8_train, drop_first=True)

y8_train = df['Target_Status']

X8_test = pd.get_dummies(df2[main_vars8], drop_first=True)
X8_test_encoded = X8_test.reindex(columns=X8_train_encoded.columns, fill_value=0)

In [ ]:
model8 = RandomForestClassifier(random_state=42)
model8.fit(X8_train_encoded, y8_train)

y8_pred = model8.predict(X8_test_encoded)

submission8 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y8_pred
})
submission8.to_csv('submission8.csv', index=False)

In [ ]:
main_vars9 = ['ID', 
                    'DRUGNAME', 'GENENAME','Disease_of_highest_status',
                    'SEQUENCE', 'BIOCLASS',
                    'Disease']

In [ ]:
X9_train = df[main_vars9]
X9_train_encoded = pd.get_dummies(X9_train, drop_first=True)

y9_train = df['Target_Status']

X9_test = pd.get_dummies(df2[main_vars9], drop_first=True)
X9_test_encoded = X9_test.reindex(columns=X9_train_encoded.columns, fill_value=0)

In [ ]:
model9 = RandomForestClassifier(random_state=42)
model9.fit(X9_train_encoded, y9_train)

y9_pred = model9.predict(X9_test_encoded)

submission9 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y9_pred
})
submission9.to_csv('submission9.csv', index=False)

In [ ]:
main_vars10 = ['ID', 
                    'DRUGNAME', 'GENENAME',
                    'Disease']

In [ ]:
X10_train = df[main_vars10]
X10_train_encoded = pd.get_dummies(X10_train, drop_first=True)

y10_train = df['Target_Status']

X10_test = pd.get_dummies(df2[main_vars10], drop_first=True)
X10_test_encoded = X10_test.reindex(columns=X10_train_encoded.columns, fill_value=0)

In [ ]:
model10 = RandomForestClassifier(random_state=42)
model10.fit(X10_train_encoded, y10_train)

y10_pred = model10.predict(X10_test_encoded)

submission10 = pd.DataFrame({
    'ID': df2['ID'], 
    'Prediction': y10_pred
})
submission10.to_csv('submission10.csv', index=False)